In [1]:
#温度预测
import os
fname = os.path.join("JenaClimate/jena_climate_2009_2016","jena_climate_2009_2016.csv")

f = open(fname)
data = f.read()
f.close()

In [3]:
lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

print(header)
print(lines)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
header

['"Date Time"',
 '"p (mbar)"',
 '"T (degC)"',
 '"Tpot (K)"',
 '"Tdew (degC)"',
 '"rh (%)"',
 '"VPmax (mbar)"',
 '"VPact (mbar)"',
 '"VPdef (mbar)"',
 '"sh (g/kg)"',
 '"H2OC (mmol/mol)"',
 '"rho (g/m**3)"',
 '"wv (m/s)"',
 '"max. wv (m/s)"',
 '"wd (deg)"']

In [6]:
#解析数据
import numpy as np

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values


In [10]:
import matplotlib.pyplot as plt
temp = float_data[:,1]#温度

plt.plot(range(len(temp)), temp)
plt.show()

In [12]:
#绘制前10天的温度时间序列
#每10分钟记录一个数据,每天144个
plt.plot(range(1440), temp[:1440])
plt.show()

需要完成以下2件事情:
- 将数据预处理为神经网络可以处理的格式,需要对每个时间序列标准化,因为温度气压位于不同范围

- 编写python生成器,以当前的浮点数数组作为输入,并从最近的数据中生成数据批量,同时生成未来的目标温度.


In [13]:
#标准化
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std


In [15]:
# 生成时间序列样本及其目标的生成器
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128,
              step=6):
    """
    
    :param data: 浮点数数据组成的原始数组
    :param lookback: 输入数据应该包括过去多少个时间步
    :param delay: 目标应该在未来多少个时间步之后
    :param min_index: data数组中的索引,用于界定需要抽取哪些时间步/用于验证集和测试分类
    :param max_index: 同上
    :param shuffle: 打乱?
    :param batch_size: 每批样本数量
    :param step: 采样周期
    """
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback

    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))

        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets


In [16]:
#准备生成器,验证集生成器和测试集生成器
lookback = 1440
step = 6
delay = 144
batch_size = 128

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step,
                      batch_size=batch_size)

val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step,
                    batch_size=batch_size)

test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=300001,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

val_steps = (300000 - 200000 - lookback) // batch_size

test_steps = (len(float_data) - 300001 - lookback) // batch_size


In [17]:
##一种基于常识的,非机器学习的方法作为基准
#计算符合常识的基准方法的MAE
def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)
        preds = samples[:, -1, -1]
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
    print(np.mean(batch_maes))

evaluate_naive_method()

1.054366071544241


In [20]:
#将mae转换成温度误差
celsius_mae = 0.29 * std[1]
celsius_mae


2.5672247338393395

In [21]:
## 一种基本的机器学习方法
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Flatten(input_shape=(lookback // step, float_data.shape[-1])))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(),loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=20,
                              validation_data=val_gen,
                              validation_steps=val_steps)



Using TensorFlow backend.


Epoch 1/20


  1/500 [..............................] - ETA: 16:47 - loss: 1.4801

 11/500 [..............................] - ETA: 1:32 - loss: 4.7555 

 20/500 [>.............................] - ETA: 51s - loss: 3.7360 

...

477/500 [===========================>..] - ETA: 0s - loss: 0.1978

483/500 [===========================>..] - ETA: 0s - loss: 0.1979

491/500 [============================>.] - ETA: 0s - loss: 0.1979

497/500 [============================>.] - ETA: 0s - loss: 0.1979

500/500 [==============================] - 8s 17ms/step - loss: 0.1979 - val_loss: 0.3361


In [23]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Training val loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()
